In [1]:
import sqlite3
import pandas as pd

In [2]:
db = sqlite3.connect('../accounts.db')

In [3]:

df = pd.read_sql_query("SELECT * FROM accounts", db)
df

,user_id,password
0,vnguyen,pbkdf2:sha256:150000$uXoonaaq$8d1ff2858afa0611...
1,brao,pbkdf2:sha256:150000$gUyNJVrA$08e184f066d63359...
2,altruista,pbkdf2:sha256:150000$K7kJZjiZ$26a2078333e67ab1...


In [7]:
## create date limit table
c = db.cursor()
c.execute('''
CREATE TABLE IF NOT EXISTS date_limits (
    user_id text NOT NULL,
    begin_date text NOT NULL,
    end_date text NOT NULL,
    FOREIGN KEY (user_id) REFERENCES accounts (user_id)
)

''')
#db.commit()

In [8]:
import datetime
import calendar

def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

today = datetime.date.today()
num_months = 12
end_date = add_months(today, num_months)

c = db.cursor()
c.execute("INSERT INTO date_limits (user_id, begin_date, end_date) VALUES (?, ?, ?)", ('vnguyen', today, end_date))

pd.read_sql_query("SELECT * FROM date_limits", db)

,user_id,begin_date,end_date
0,vnguyen,2020-10-06,2021-10-06


In [6]:
# delete table
c.execute("DROP TABLE date_limits")

In [69]:
# update date limits

new_num_months = 1
end_date = add_months(today, new_num_months)

u_id = 'vnguyen'

c.execute("UPDATE date_limits SET end_date=(?) WHERE user_id=(?) ", (end_date, u_id))

pd.read_sql_query("SELECT * FROM date_limits", db)

,date_id,user_id,begin_date,end_date


In [12]:
# update brao
end_date = add_months(today, 12)
c.execute("INSERT INTO date_limits (user_id, begin_date, end_date) VALUES (?, ?, ?)", ('brao', today, end_date))

In [13]:
# update altruista
end_date = add_months(today, 1)
c.execute("INSERT INTO date_limits (user_id, begin_date, end_date) VALUES (?, ?, ?)", ('altruista', today, end_date))

In [14]:
pd.read_sql_query("SELECT * FROM date_limits", db)

,user_id,begin_date,end_date
0,vnguyen,2020-10-06,2021-10-06
1,brao,2020-10-06,2021-10-06
2,altruista,2020-10-06,2020-11-06


In [15]:
db.commit()

In [16]:
db.close()